<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/convnet_cats_%26_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import library
# /content/cats_and_dogs_subset/12.jpg
import os, shutil
print('Operating system: ', os.name)
print('Get current working directory: ', os.getcwd()) 

Operating system:  posix
Get current working directory:  /content


In [0]:
# Locate data
original_dataset_dir = '/content/cats_and_dogs_all'
print(os.listdir('cats_and_dogs_subset'))

In [0]:
# Create directory

base_dir = '/content/cats_and_dogs_subset'
os.mkdir(base_dir)

# Create directories top level
# Training
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

# Validating
validation_dir =  os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

# Testing
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Create directories second level
# Train
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Validation
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Testing
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [0]:
# Copies images to assigned folders

# Cats / Training
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(train_cats_dir, fname)
  shutil.copyfile(src, dst)

# Cats / Validation
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(validation_cats_dir, fname)
  shutil.copyfile(src, dst)

# Cats / Testing
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(test_cats_dir, fname)
  shutil.copyfile(src, dst)

# Dogs / Training
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(train_dogs_dir, fname)
  shutil.copyfile(src, dst)

# Dogs / Validation
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(validation_dogs_dir, fname)
  shutil.copyfile(src, dst)

# Dogs / Testing
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(test_dogs_dir, fname)
  shutil.copyfile(src, dst)

In [0]:
# Review files

# Training
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))

# Validation
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

# Testing
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 0
total training dog images: 0
total validation cat images: 0
total validation dog images: 0
total test cat images: 0
total test dog images: 0


In [0]:
print(os.path.abspath('1.jpg'))

/content/1.jpg
